In [190]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from datetime import datetime

In [191]:
STANDINGS_DIR = "data/standings"

In [192]:
def get_current_nba_season():
    now = datetime.now()
    month = now.month
    year = now.year
    if (month >= 10): #If october, then in the next year's season (we call 2024-25 season "2025" in the files)
        return str(year+1)
    else: #If before october, then in current year 
        return str(year)

In [193]:
season_year = get_current_nba_season()
season_year

'2025'

In [194]:
standings = os.listdir(STANDINGS_DIR)
standings = [x for x in standings if season_year in x] # Filter current szn year
standings = [os.path.join(STANDINGS_DIR, f) for f in standings if f.endswith(".html")] #join together full path to standings
if len(standings) == 0:
    print(f"You have no files for the {year} season, may have to scrape from get_data")
standings

['data/standings/NBA_2025_games-april.html',
 'data/standings/NBA_2025_games-december.html',
 'data/standings/NBA_2025_games-february.html',
 'data/standings/NBA_2025_games-january.html',
 'data/standings/NBA_2025_games-march.html',
 'data/standings/NBA_2025_games-november.html',
 'data/standings/NBA_2025_games-october.html']

In [195]:
def parse_month(standing, drop_past = True):
    with open(standing, encoding = 'utf-8') as f:
        html = f.read()
    soup = BeautifulSoup(html)
    tbl = soup.find("table", {"id": "schedule"})
    [s.decompose() for s in tbl.select("tr.thead")] #thead class is strictly repeated headers, remove them so they don't show
    
    df = pd.read_html(StringIO(str(tbl)))[0] #read_html returns list of df's so index
    #Rename box-score column, this is how we'll know if it is a future game or not (only finished games have them)
    df = df.rename(columns={'Unnamed: 6': 'Box Score'})
    #Drop all extra columns to the right of box-score 
    col_index = df.columns.get_loc('Box Score') 
    df = df.iloc[:, :col_index + 1]
    #If we only want future games, take only the rows with NAN box-scores
    if drop_past:
        df = df[df['Box Score'].isna()]
    return df

In [196]:
all_standings = []
#Loop through all the months of current year and process tables
for standing in standings:
    df = parse_month(standing)
    all_standings.append(df)
#Combine all the months into one table
future_games = pd.concat(all_standings)
#Re-format date so it's cleaner, then sort by date
future_games['Date'] = pd.to_datetime(future_games['Date'], format='%a, %b %d, %Y')
future_games['Date'] = future_games['Date'].dt.strftime('%Y-%m-%d')
future_games = future_games.sort_values(by='Date')
future_games = future_games.reset_index(drop=True)

In [197]:
future_games

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Box Score
0,2025-01-29,8:00p,Dallas Mavericks,NaN,New Orleans Pelicans,NaN,NaN
1,2025-01-29,7:00p,Brooklyn Nets,NaN,Charlotte Hornets,NaN,NaN
2,2025-01-29,7:00p,Detroit Pistons,NaN,Indiana Pacers,NaN,NaN
3,2025-01-29,7:00p,Toronto Raptors,NaN,Washington Wizards,NaN,NaN
4,2025-01-29,7:30p,Chicago Bulls,NaN,Boston Celtics,NaN,NaN
...,...,...,...,...,...,...,...
537,2025-04-13,1:00p,New York Knicks,NaN,Brooklyn Nets,NaN,NaN
538,2025-04-13,1:00p,Charlotte Hornets,NaN,Boston Celtics,NaN,NaN
539,2025-04-13,1:00p,Orlando Magic,NaN,Atlanta Hawks,NaN,NaN
540,2025-04-13,3:30p,Los Angeles Clippers,NaN,Golden State Warriors,NaN,NaN


In [198]:
future_games.to_csv('future_test.csv')
#future_games.to_csv(f'data/future_games/future_games_{season_year}.csv')